In [1]:
import cv2
from matplotlib import pyplot as plt
import os
import torch
import numpy as np
from queue import PriorityQueue
import time
import psutil
import sys

In [2]:
def measure_processing_power(func, *args, **kwargs):
    """
    Measure CPU and memory usage during the execution of a function.

    Args:
        func (callable): The function to measure.
        *args: Positional arguments for the function.
        **kwargs: Keyword arguments for the function.

    Returns:
        result: The result of the function execution.
        stats: A dictionary containing CPU and memory usage stats.
    """
    # Record initial stats
    process = psutil.Process()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss
    start_time = time.perf_counter()

    # Run the function
    result = func(*args, **kwargs)

    # Record final stats
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss
    end_time = time.perf_counter()

    # Calculate deltas
    cpu_usage = end_cpu - start_cpu
    mem_usage = end_mem - start_mem
    duration = end_time - start_time

    stats = {
        "CPU_Usage (%)": cpu_usage,
        "Memory_Usage (bytes)": mem_usage,
        "Duration (seconds)": duration
    }

    return result, stats

In [3]:
def draw_detections(image, detections, label_map=None, color=(0, 255, 0), thickness=5):
    """
    Draw bounding boxes and labels on an image to visualize detections.

    Args:
        image (ndarray): The image on which to draw detections.
        detections (list): List of detections. Each detection is a dictionary containing:
            - 'x_min': Minimum x-coordinate of the bounding box.
            - 'y_min': Minimum y-coordinate of the bounding box.
            - 'x_max': Maximum x-coordinate of the bounding box.
            - 'y_max': Maximum y-coordinate of the bounding box.
            - 'label' (optional): Label or class ID of the detection.
        label_map (dict, optional): A mapping from class IDs to human-readable labels.
        color (tuple): Color of the bounding box (default: green).
        thickness (int): Thickness of the bounding box lines (default: 2).

    Returns:
        annotated_image (ndarray): The image with drawn detections.
    """
    annotated_image = image.copy()

    for detection in detections:
        x_min = int(detection['x_min'])
        y_min = int(detection['y_min'])
        x_max = int(detection['x_max'])
        y_max = int(detection['y_max'])
        label = detection.get('label', None)

        # Draw the bounding box
        cv2.rectangle(annotated_image, (x_min, y_min), (x_max, y_max), color, thickness)

        # Draw the label if available
        if label is not None:
            label_text = str(label_map[label]) if label_map and label in label_map else str(label)
            cv2.putText(
                annotated_image,
                label_text,
                (x_min, y_min - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                2,
                color,
                5
            )

    return annotated_image

def save_yolo_labels( labels, output_dir, image_name ):
    os.makedirs(f"{output_dir}/labels", exist_ok=True)

    label_path = f"{output_dir}/labels/{image_name}.txt"
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")
    print(f"Saved YOLO label file: {label_path}")

def save_annotated_image( image, detections, output_dir, image_name, suffix="" ):
    os.makedirs(f"{output_dir}/images", exist_ok=True)
    annotated_image = draw_detections(image, detections)
    annotated_image_path = f"{output_dir}/images/{image_name}_result{suffix}.jpg"
    cv2.imwrite(annotated_image_path, annotated_image)
    print(f"Saved annotated image: {annotated_image_path}")

In [4]:
def qr_code_detector(image,):
    """
    Detect QR codes using OpenCV and save results in YOLO format.

    Args:
        image_path (str): Path to the input image.
        output_dir (str): Directory to save results (images and labels).
        save_annotated (bool): If True, save the annotated image with QR code detections.
    """
    
    # Load the image
    h, w = image.shape[:2]
    
    # Initialize QR Code detector
    qr_detector = cv2.QRCodeDetector()

    # Detect and decode QR codes
    data, _, vertices, _ = qr_detector.detectAndDecodeMulti(image)

    detections = []  # Store detection information for drawing
    labels = []      # Store YOLO-format labels

    if vertices is not None:
        for i, points in enumerate(vertices):
            # Calculate bounding box coordinates
            x_min = points[:, 0].min()
            y_min = points[:, 1].min()
            x_max = points[:, 0].max()
            y_max = points[:, 1].max()

            # Append to detections for visualization
            detections.append({
                'x_min': x_min,
                'y_min': y_min,
                'x_max': x_max,
                'y_max': y_max,
                'label': f"QR {i}"  # Optional label for visualization
            })

            # Calculate YOLO-format labels
            x_center = (x_min + x_max) / 2 / w  # Normalize by image width
            y_center = (y_min + y_max) / 2 / h  # Normalize by image height
            width = (x_max - x_min) / w         # Normalize width
            height = (y_max - y_min) / h        # Normalize height

            labels.append([0, x_center, y_center, width, height])  # Assuming class_id = 0 for QR codes
    else:
        print(f"No QR codes detected in the image")

    return detections, labels

In [ ]:
# Example Usage
test_dir = "../data/test/samples/"
cv2_results_dir = "../data/test/results/cv2_detector/"

# Run QR code detection and save results for each image in the test directory
for filename in os.listdir(test_dir):
    if filename.endswith((".jpg", ".png", ".jpeg", ".JPG")) and "out" not in filename and filename[0] != ".":
        filepath = os.path.join(test_dir, filename)
        image_name = os.path.splitext(os.path.basename(filepath))[0]
        print(f"Processing: {filepath}")
        image = cv2.imread(filepath)
        if image is None:
            raise ValueError(f"Error: Could not load image at {filepath}")
        cv2_detections, cv2_labels = qr_code_detector(image)
        print(cv2_detections)
        save_yolo_labels(cv2_labels, cv2_results_dir, image_name)
        save_annotated_image( image, cv2_detections, cv2_results_dir, image_name)

Processing: ../data/test/samples/test_light_130.JPG
[{'x_min': 1908.8589, 'y_min': 3035.5122, 'x_max': 1994.0, 'y_max': 3157.0, 'label': 'QR 0'}, {'x_min': 1903.0, 'y_min': 2585.0, 'x_max': 1992.0, 'y_max': 2708.0, 'label': 'QR 1'}, {'x_min': 2316.0, 'y_min': 1667.0, 'x_max': 3016.0, 'y_max': 2594.0, 'label': 'QR 2'}]
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_light_130.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_130_result.jpg
Processing: ../data/test/samples/test_light_120.JPG
[{'x_min': 1776.0, 'y_min': 2511.7073, 'x_max': 1873.1279, 'y_max': 2644.0, 'label': 'QR 0'}, {'x_min': 2317.0, 'y_min': 3028.6, 'x_max': 2440.049, 'y_max': 3183.338, 'label': 'QR 1'}]
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_light_120.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_120_result.jpg
Processing: ../data/test/samples/test_110.JPG
[{'x_min': 1717.1102, 'y_min': 3058.7502, 'x_ma

In [6]:
def yolo_preprocessing(image, input_size=(640, 640)):
    """
    Preprocesses an image for YOLO inference.
    
    Args:
        image (np.array): Image array.
        input_size (tuple): Desired input size (width, height).
        
    Returns:
        torch.Tensor: Preprocessed image tensor ready for YOLO inference.
    """
    
    # Get original dimensions
    original_height, original_width = image.shape[:2]
    
    # Resize while keeping aspect ratio (letterboxing)
    scale = min(input_size[0] / original_width, input_size[1] / original_height)
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    
    # Create a new blank image with the desired input size and center the resized image
    padded_image = np.full((input_size[1], input_size[0], 3), 114, dtype=np.uint8)
    x_offset = (input_size[0] - new_width) // 2
    y_offset = (input_size[1] - new_height) // 2
    padded_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image
    
    # Convert BGR to RGB
    padded_image = padded_image[:, :, ::-1]
    
    # Normalize to [0, 1]
    normalized_image = padded_image / 255.0
    
    # Convert to channels-first format
    transposed_image = normalized_image.transpose(2, 0, 1)  # HWC to CHW
    
    # Convert to torch tensor and add batch dimension
    input_tensor = torch.from_numpy(transposed_image).unsqueeze(0).float()
    
    return input_tensor

def yolov5n_detector(image, model, input_size=(640, 640), conf_threshold=0.25):
    """
    Perform inference with YOLOv5, including preprocessing and result postprocessing.
    
    Args:
        image (np.array): Input image.
        model: Loaded YOLOv5 model.
        input_size (tuple): YOLO model input size.
        conf_threshold (float): Confidence threshold for filtering detections.
        
    Returns:
        list: Detections with coordinates in the original image space.
        list: YOLO labels in normalized format.
    """
    # Get original dimensions
    original_height, original_width = image.shape[:2]
    
    # Preprocess image for YOLO
    input_tensor = yolo_preprocessing(image, input_size)

    # Perform inference
    results = model(input_tensor)
    
    # Extract detections (results.xyxy[0] contains the predictions)
    detections = []
    labels = []
    
    for row in results[0]:  # Loop through detections in the first image (batch size 1)
        # row = [x_min, y_min, x_max, y_max, confidence, class]
        x_min, y_min, x_max, y_max, conf, class_id = row.tolist()
        
        if conf < conf_threshold:
            continue  # Skip detections below confidence threshold

        # Remove padding and scale back to the original image dimensions
        scale = min(input_size[0] / original_width, input_size[1] / original_height)
        x_offset = (input_size[0] - scale * original_width) / 2
        y_offset = (input_size[1] - scale * original_height) / 2

        x_min = (x_min - x_offset) / scale
        x_max = (x_max - x_offset) / scale
        y_min = (y_min - y_offset) / scale
        y_max = (y_max - y_offset) / scale

        detections.append({
            'x_min': x_min,
            'y_min': y_min,
            'x_max': x_max,
            'y_max': y_max,
            'label': int(class_id),
            'confidence': conf
        })

        # Normalize for YOLO format
        x_center = (x_min + x_max) / 2 / original_width
        y_center = (y_min + y_max) / 2 / original_height
        width = (x_max - x_min) / original_width
        height = (y_max - y_min) / original_height

        labels.append([int(class_id), x_center, y_center, width, height])

    return detections, labels

In [7]:



# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../data/models/yolov5n_best.pt')

# Verify the model is loaded
print("Model loaded successfully!")

# Input and output paths
test_dir = "../data/test/samples/"
yolo_results_dir = "../data/test/results/yolov5n/"

# Run QR code detection and save results for each image in the test directory
for filename in os.listdir(test_dir):
    if filename.endswith((".jpg", ".png", ".jpeg", ".JPG")) and "out" not in filename and filename[0] != ".":
        filepath = os.path.join(test_dir, filename)
        image_name = os.path.splitext(os.path.basename(filepath))[0]
        print(f"Processing: {filepath}")
        image = cv2.imread(filepath)
        if image is None:
            raise ValueError(f"Error: Could not load image at {filepath}")
        yolo_detections, yolo_labels = yolov5n_detector(image, model)
        print(yolo_detections)
        save_yolo_labels(yolo_labels, yolo_results_dir, image_name)
        save_annotated_image( image, cv2_detections, cv2_results_dir, image_name)

Using cache found in /Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-12 Python-3.11.11 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Model loaded successfully!
Processing: ../data/test/samples/test_light_130.JPG
[{'x_min': 1936.5787376403807, 'y_min': 2639.3124504089355, 'x_max': -632.6686206579208, 'y_max': 92.81432790756226, 'label': 0, 'confidence': 0.43776223063468933}, {'x_min': 2475.103399658203, 'y_min': 2627.3020500183106, 'x_max': -601.2082544088363, 'y_max': 119.36729578971863, 'label': 0, 'confidence': 0.7376148700714111}, {'x_min': 2476.57147064209, 'y_min': 2626.1096168518065, 'x_max': -606.0760911226272, 'y_max': 119.82423799037933, 'label': 0, 'confidence': 0.6636061072349548}, {'x_min': 1939.0295173645018, 'y_min': 2640.0829833984376, 'x_max': -631.6699697971344, 'y_max': 88.58497610092162, 'label': 0, 'confidence': 0.45955604314804077}, {'x_min': 2476.265599822998, 'y_min': 2627.4099082946777, 'x_max': -600.3354041576385, 'y_max': 123.14218225479125, 'label': 0, 'confidence': 0.739990234375}, {'x_min': 2473.6816314697267, 'y_min': 3150.0608825683594, 'x_max': -620.8141006708145, 'y_max': 94.98386921

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


[{'x_min': 1820.7416725158691, 'y_min': 2562.726537322998, 'x_max': -628.510076379776, 'y_max': 95.11347448825836, 'label': 0, 'confidence': 0.6498938798904419}, {'x_min': 1820.3562698364258, 'y_min': 2565.519140625, 'x_max': -628.6687145233154, 'y_max': 99.12922911643982, 'label': 0, 'confidence': 0.6676197052001953}, {'x_min': 1816.4589363098144, 'y_min': 3085.80676574707, 'x_max': -644.6813106536865, 'y_max': 78.48900904655456, 'label': 0, 'confidence': 0.48762115836143494}, {'x_min': 1815.1390342712402, 'y_min': 3082.4095024108888, 'x_max': -643.9117352128029, 'y_max': 82.9975224494934, 'label': 0, 'confidence': 0.5825177431106567}, {'x_min': 2383.1566703796384, 'y_min': 3113.6031509399413, 'x_max': -607.0439301967621, 'y_max': 130.8005028963089, 'label': 0, 'confidence': 0.6522613167762756}, {'x_min': 2385.0638008117676, 'y_min': 3114.6678428649902, 'x_max': -598.4239583730697, 'y_max': 128.983824634552, 'label': 0, 'confidence': 0.6414011120796204}, {'x_min': 2383.6330444335936, 

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


[{'x_min': 1731.484590911865, 'y_min': 2558.031433868408, 'x_max': -613.113741517067, 'y_max': 116.83451585769653, 'label': 0, 'confidence': 0.7147844433784485}, {'x_min': 1729.3761795043945, 'y_min': 2559.4567428588866, 'x_max': -612.0259577751159, 'y_max': 119.43447740077971, 'label': 0, 'confidence': 0.8462671041488647}, {'x_min': 2344.346212005615, 'y_min': 2543.6021209716796, 'x_max': -589.5621310472488, 'y_max': 138.55016090869904, 'label': 0, 'confidence': 0.6230184435844421}, {'x_min': 2343.146152496338, 'y_min': 2544.426583099365, 'x_max': -587.5884488582611, 'y_max': 146.79247555732726, 'label': 0, 'confidence': 0.6669336557388306}, {'x_min': 1729.0196479797362, 'y_min': 2560.148833465576, 'x_max': -611.4468664169311, 'y_max': 124.65044474601746, 'label': 0, 'confidence': 0.8546955585479736}, {'x_min': 2344.490567779541, 'y_min': 2551.093913269043, 'x_max': -586.4007991790771, 'y_max': 143.78597717285155, 'label': 0, 'confidence': 0.5026654005050659}, {'x_min': 1724.037467193

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


[{'x_min': 1712.829467010498, 'y_min': 2563.1544296264647, 'x_max': -591.2219585895538, 'y_max': 134.58219008445738, 'label': 0, 'confidence': 0.7426589131355286}, {'x_min': 1711.8274200439453, 'y_min': 2559.380751800537, 'x_max': -593.915947151184, 'y_max': 132.43774931430815, 'label': 0, 'confidence': 0.6310000419616699}, {'x_min': 2400.6057426452635, 'y_min': 2548.642044067383, 'x_max': -559.5943745613098, 'y_max': 171.2446413516998, 'label': 0, 'confidence': 0.5118107795715332}, {'x_min': 2400.8977226257325, 'y_min': 2544.7814804077148, 'x_max': -567.4121589660645, 'y_max': 172.8445731639862, 'label': 0, 'confidence': 0.35732191801071167}, {'x_min': 1712.3977615356446, 'y_min': 2566.7597831726075, 'x_max': -590.1650802612304, 'y_max': 139.02463688850403, 'label': 0, 'confidence': 0.42194175720214844}, {'x_min': 1712.4211853027343, 'y_min': 2563.778155517578, 'x_max': -591.7720255851746, 'y_max': 140.037442445755, 'label': 0, 'confidence': 0.6913629174232483}, {'x_min': 2399.4353713

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [8]:
def format_detections_for_transform(detections, image_shape):
    """
    Reorders detections into srcPoints corresponding to the quadrants 
    (top-left, top-right, bottom-left, bottom-right) for homography calculation.

    Args:
        detections (list): List of dictionaries with bounding box details.
        image_shape (tuple): Shape of the image as (height, width).

    Returns:
        list: List of four (x, y) source points in the correct order.
    """
    h, w = image_shape[:2]

    # Initialize lists for each quadrant
    top_left, top_right, bottom_left, bottom_right = [], [], [], []

    tl_point = (-1,-1)
    br_point = (-1,-1)

    for detection in detections:
        x_min = detection['x_min']
        y_min = detection['y_min']
        x_max = detection['x_max']
        y_max = detection['y_max']
        center_x = (x_min + x_max) / 2
        center_y = (y_min + y_max) / 2

        # Assign to quadrants
        if center_x < w / 2 and center_y < h / 2:
            top_left.append((center_x, center_y))
            tl_point = (x_min, y_min)
        elif center_x >= w / 2 and center_y < h / 2:
            top_right.append((center_x, center_y))
        elif center_x < w / 2 and center_y >= h / 2:
            bottom_left.append((center_x, center_y))
        else:
            bottom_right.append((center_x, center_y))
            br_point = (x_max, y_max)

    # Verify we have one point per quadrant
    if len(top_left) != 1 or len(top_right) != 1 or len(bottom_left) != 1 or len(bottom_right) != 1:
        raise ValueError("Four QR codes (one in each quadrant) are required for the transform.")

    # Return points in the order: top-left, top-right, bottom-left, bottom-right
    out_points = [top_left[0], top_right[0], bottom_right[0], bottom_left[0]]
    return out_points, tl_point, br_point

In [9]:
filepath = "../data/targets/output/scoring_target_qr_5_h.jpg"
# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../data/models/yolov5n_best.pt')

image_name = os.path.splitext(os.path.basename(filepath))[0]
print(f"Processing: {filepath}")
image = cv2.imread(filepath)
if image is None:
    raise ValueError(f"Error: Could not load image at {filepath}")


result, stats = measure_processing_power(qr_code_detector, image)
cv2_detections, cv2_labels = result
print(cv2_detections)
print(stats)
save_annotated_image( image, cv2_detections, "../data/targets/sandbox_run/cv2/", image_name)
result, stats = measure_processing_power(yolov5n_detector, image, model)
yolo_detections, yolo_labels = result
print(yolo_detections)
print(stats)
save_annotated_image( image, yolo_detections, "../data/targets/sandbox_run/yolo/", image_name)

dst_points, _, _ = format_detections_for_transform(cv2_detections, image.shape)

Using cache found in /Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-12 Python-3.11.11 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


Processing: ../data/targets/output/scoring_target_qr_5_h.jpg
[{'x_min': 2719.0, 'y_min': 1970.0, 'x_max': 3229.0, 'y_max': 2479.0, 'label': 'QR 0'}, {'x_min': 2713.0, 'y_min': 63.0, 'x_max': 3236.0, 'y_max': 586.0, 'label': 'QR 1'}, {'x_min': 70.0, 'y_min': 70.0, 'x_max': 579.0, 'y_max': 579.0, 'label': 'QR 2'}, {'x_min': 77.89289, 'y_min': 1978.0, 'x_max': 571.0, 'y_max': 2473.1882, 'label': 'QR 3'}]
{'CPU_Usage (%)': 166.2, 'Memory_Usage (bytes)': -343277568, 'Duration (seconds)': 0.732163708016742}
Saved annotated image: ../data/targets/sandbox_run/cv2//images/scoring_target_qr_5_h_result.jpg
[]
{'CPU_Usage (%)': 391.1, 'Memory_Usage (bytes)': 47759360, 'Duration (seconds)': 0.11598699999740347}
Saved annotated image: ../data/targets/sandbox_run/yolo//images/scoring_target_qr_5_h_result.jpg


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [10]:
def find_four_point_transform(srcPoints, dstPoints):
    """Solves for and returns a perspective transform.

    Each source and corresponding destination point must be at the
    same index in the lists.

    Do not use the following functions (you will implement this yourself):
        cv2.findHomography
        cv2.getPerspectiveTransform
    Hint: You will probably need to use least squares to solve this.
    Args:
        srcPoints (list): List of four (x,y) source points
        dstPoints (list): List of four (x,y) destination points
    Returns:
        numpy.array: 3 by 3 homography matrix of floating point values
    """

    # Generate pair of equations for each point
    p = srcPoints
    t = dstPoints
    
    A = np.zeros((8, 9))
    
    for i in range(len(p)):
        A[2 * i + 0] = [p[i][0], p[i][1], 1, 0, 0, 0, -t[i][0] * p[i][0], -t[i][0] * p[i][1], -t[i][0]]
        A[2 * i + 1] = [0, 0, 0, p[i][0], p[i][1], 1,  -t[i][1] * p[i][0], -t[i][1] * p[i][1], -t[i][1]]
    
    # Execute singular value decomposition
    u, s, vh = np.linalg.svd(A)
    
    # Minimizing by taking last column of V, generating homography
    homography = np.ones((3,3))
    count = 0
    for i in range(3):
        for j in range(3):
            homography[i,j] = vh[8,count]
            count += 1
    return homography


In [11]:
def validate_and_redetect(image, detections, template, pyramid_levels=3, threshold=0.0):
    """
    Validate initial detections and attempt redetection if validation fails.

    Args:
        image (ndarray): The input image.
        detections (list): List of initial detections with bounding box coordinates.
        template (ndarray): Template of the QR code to search for.
        pyramid_levels (int): Number of pyramid levels for validation and redetection.
        threshold (float): Minimum confidence for template matching.

    Returns:
        updated_detections (list): List of validated or redetected bounding boxes.
    """
    updated_detections = []
    h, w = image.shape[:2]

    # Create Laplacian pyramid for the image
    image_pyramid = [image]
    for _ in range(pyramid_levels - 1):
        image_pyramid.append(cv2.pyrDown(image_pyramid[-1]))

    for detection in detections:
        # Extract ROI around the detection
        x_min, y_min, x_max, y_max = (
            int(detection['x_min']),
            int(detection['y_min']),
            int(detection['x_max']),
            int(detection['y_max']),
        )

        roi = image[y_min:y_max, x_min:x_max]
        resized_template = cv2.resize(template, (roi.shape[1], roi.shape[0]))

        # Validate using template matching within the ROI
        result = cv2.matchTemplate(roi, resized_template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        print("match result: ",max_val)

        if max_val >= threshold:
            # If validation succeeds, adjust coordinates to the original scale
            updated_detections.append({
                'x_min': x_min,
                'y_min': y_min,
                'x_max': x_max,
                'y_max': y_max,
                'confidence': max_val
            })

    return updated_detections

In [12]:
filepath = "../data/test/samples/test_marks.JPG"
# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../data/models/yolov5n_best.pt')

image_name = os.path.splitext(os.path.basename(filepath))[0]
print(f"Processing: {filepath}")
image = cv2.imread(filepath)
if image is None:
    raise ValueError(f"Error: Could not load image at {filepath}")


result, stats = measure_processing_power(qr_code_detector, image)
cv2_detections, cv2_labels = result
print(cv2_detections)
print(stats)
save_annotated_image( image, cv2_detections, "../data/targets/sandbox_run/cv2", image_name)

#print(rescan_image(image, cv2_detections, threshold=0.1))
template = cv2.imread("../data/qr_codes/qr_tr.png")
result, stats = measure_processing_power(validate_and_redetect, image, cv2_detections, template, pyramid_levels=3, threshold=0.0)
print(stats)
filtered_detections = result.copy()
save_annotated_image(image, filtered_detections, "../data/tests/sandbox_run/cv2", image_name, suffix="_tmatch")

src_points, tl_point, br_point = format_detections_for_transform(filtered_detections, image.shape)

Using cache found in /Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-12 Python-3.11.11 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


Processing: ../data/test/samples/test_marks.JPG
[{'x_min': 3360.4148, 'y_min': 1623.0, 'x_max': 3615.5322, 'y_max': 1899.5979, 'label': 'QR 0'}, {'x_min': 3317.0, 'y_min': 2600.667, 'x_max': 3557.0, 'y_max': 2894.2947, 'label': 'QR 1'}, {'x_min': 2364.7778, 'y_min': 1623.0138, 'x_max': 2547.1968, 'y_max': 1855.8821, 'label': 'QR 2'}, {'x_min': 1993.0, 'y_min': 21.0, 'x_max': 5709.0, 'y_max': 1544.0, 'label': 'QR 3'}, {'x_min': 2328.4646, 'y_min': 2476.3237, 'x_max': 2507.452, 'y_max': 2722.0, 'label': 'QR 4'}]
{'CPU_Usage (%)': 104.8, 'Memory_Usage (bytes)': 66699264, 'Duration (seconds)': 3.1436754589667544}
Saved annotated image: ../data/targets/sandbox_run/cv2/images/test_marks_result.jpg
match result:  (0.10503210872411728, 0.10503210872411728, (0, 0), (0, 0))
match result:  (0.09911054372787476, 0.09911054372787476, (0, 0), (0, 0))
match result:  (0.013012969866394997, 0.013012969866394997, (0, 0), (0, 0))
match result:  (-0.14484745264053345, -0.14484745264053345, (0, 0), (0, 0))

In [13]:
print(src_points)
print(dst_points)
homography = find_four_point_transform(src_points, dst_points)

In [14]:
def correspondences(x,y,eig):
    
    num_x = eig[0][0] * x + eig[0][1] * y + eig[0][2]
    den_x = eig[2][0] * x + eig[2][1] * y + eig[2][2]
    
    num_y = eig[1][0] * x + eig[1][1] * y + eig[1][2]
    den_y = eig[2][0] * x + eig[2][1] * y + eig[2][2]
    
    x_t = num_x / den_x
    y_t = num_y / den_y
    
    return x_t, y_t

def crop_to_corners(image, tl_point, br_point, eig):


    # transform corners
    x_tl, y_tl = correspondences(tl_point[0], tl_point[1], eig)
    x_br, y_br = correspondences(br_point[0], br_point[1], eig)
    x_tl, y_tl = int(x_tl), int(y_tl)
    x_br, y_br = int(x_br), int(y_br)

    # Crop the image
    cropped_image = image[y_tl:y_br, x_tl:x_br]

    return cropped_image


def crop_to_qr_corners(image, detections, eig):
    """
    Crops the image from the top-left QR code corner to the bottom-right QR code corner.

    Args:
        image (ndarray): The input image to be cropped.
        detections (list): List of dictionaries with bounding box details for QR codes.
        eig (ndarray): Homography matrix.

    Returns:
        ndarray: Cropped image.
    """
    # Ensure there are at least two QR codes detected
    if len(detections) < 2:
        raise ValueError("At least two QR codes (top-left and bottom-right) are required to crop the image.")

    # Find the top-left and bottom-right QR codes
    src_points = format_detections_for_transform(detections, image.shape)
    top_left = src_points[0].astype(int)  # Top-left QR code
    bottom_right = src_points[2].astype(int)  # Bottom-right QR code

    # Transform the top-left and bottom-right points using the homography matrix
    x_tl, y_tl = correspondences(top_left[0], top_left[1], eig)
    x_br, y_br = correspondences(bottom_right[0], bottom_right[1], eig)

    # Ensure the points are integers for cropping
    pad = 200
    x_tl, y_tl = int(x_tl), int(y_tl)
    x_br, y_br = int(x_br), int(y_br)

    # Crop the image
    cropped_image = image[y_tl:y_br, x_tl:x_br]

    return cropped_image

In [15]:
warped = cv2.warpPerspective(image, homography, (image.shape[1],image.shape[0]))
warped_and_cropped = crop_to_corners(warped, tl_point, br_point, homography)

In [16]:
output_size = (110,85)
cv2.imwrite("../data/tests/sandbox_run/cv2/images/output_warped_cropped.jpg",warped_and_cropped)

True

# Sandbox

In [19]:
test_dir = "../data/test/samples/"
output_dir = "../data/test/python/sandbox_run/cv2"

# Run QR code detection and save results for each image in the test directory
for file in os.listdir(test_dir):
    filepath = os.path.join(test_dir, file)
    image_name = os.path.splitext(os.path.basename(filepath))[0]
    print(f"Processing: {filepath}")
    image = cv2.imread(filepath)
    if image is None:
        raise ValueError(f"Error: Could not load image at {filepath}")

    # initial detection step
    cv2_detections, cv2_labels = qr_code_detector(image)
    print(cv2_detections)
    save_annotated_image( image, cv2_detections, output_dir, image_name)

    # template matching validation
    template = cv2.imread("../data/qr_codes/qr_tr.png")
    filtered_detections = validate_and_redetect(image, cv2_detections, template, pyramid_levels=3, threshold=-0.01)
    save_annotated_image(image, filtered_detections, output_dir, image_name, suffix="_tmatch")

    # get detections
    if len(filtered_detections) == 4:
        src_points, tl_point, br_point = format_detections_for_transform(filtered_detections, image.shape)
    else:
        print(f"Error: detections need to find 4 codes but found {len(filtered_detections)}")
        #sys.exit(1)
        continue

    # get homography
    homography = find_four_point_transform(src_points, dst_points)

    # warp based on homography and crop to tl corner and br corner
    warped = cv2.warpPerspective(image, homography, (image.shape[1],image.shape[0]))
    warped_and_cropped = crop_to_corners(warped, tl_point, br_point, homography)
    file_string = f"{output_dir}/images/{image_name}_output.jpg"
    cv2.imwrite(file_string,warped_and_cropped)
    print(f"saved to {file_string}")

Processing: ../data/test/samples/test_light_130.JPG
[{'x_min': 1908.8589, 'y_min': 3035.5122, 'x_max': 1994.0, 'y_max': 3157.0, 'label': 'QR 0'}, {'x_min': 1903.0, 'y_min': 2585.0, 'x_max': 1992.0, 'y_max': 2708.0, 'label': 'QR 1'}, {'x_min': 2316.0, 'y_min': 1667.0, 'x_max': 3016.0, 'y_max': 2594.0, 'label': 'QR 2'}]
Saved annotated image: ../data/test/python/sandbox_run/cv2/images/test_light_130_result.jpg
match result:  (0.02540392428636551, 0.02540392428636551, (0, 0), (0, 0))
match result:  (-0.0025577745400369167, -0.0025577745400369167, (0, 0), (0, 0))
match result:  (-0.0798826515674591, -0.0798826515674591, (0, 0), (0, 0))
Saved annotated image: ../data/test/python/sandbox_run/cv2/images/test_light_130_result_tmatch.jpg
Error: detections need to find 4 codes but found 2
Processing: ../data/test/samples/test_light_120.JPG
[{'x_min': 1776.0, 'y_min': 2511.7073, 'x_max': 1873.1279, 'y_max': 2644.0, 'label': 'QR 0'}, {'x_min': 2317.0, 'y_min': 3028.6, 'x_max': 2440.049, 'y_max': 3